# Rational Transfer Functions and the Power of Dynamic Representation
# On the Limitations of OLS Regression with Lagged Variables
# How far will lagged variable regressions go?

Every budding modeler at some early point stumbles upon the apparent predictive power of regressing on lagged dependent variables. That it happens long before any formal coursework on the subject is because it is so intuitive. To borrow from [Tobler's first law of geography](https://en.wikipedia.org/wiki/Tobler%27s_first_law_of_geography), "everything is related to everything else, but near things are more related than distant things," a quote which surely holds as true for time as it does for space. And of all the complex factors that come together to create a variable, only a fraction of which we may measure or fully understand, we can rely on the lagged value to be of a similar formula and hense lagged dependent variables encapsulate what we do not know. This is the essense of the reduced dynamic form, and the purpose of this article is to test its limits especially when paired with Ordinary Least Squares regression.

Aside from being props to boost a 1-step-ahead R-squared, adding lagged variables takes a model from mapping equation to stochastic process. Suppose, for $z_t = y_t - f(x_t)$ for s

## Dynamic challenge: convert a nonlinear, nonstationary, structural time-series model

There's a time-series model I've been studying because its nonlinearity, nonstationarity and its encapsulation of latent hypothetical concepts into regression variables. It is the fitness-fatigue model of atheletic performance and has the form

$$
p_t = \mu + k_1 \sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_1}\right) - k_2 \sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_2}\right) + \epsilon_t,
$$
where $p_t$ is a numeric measure of (athletic) performance, $w_t$ is training "dose" (i.e., time-weighted training intensity) occuring in time period $t$, and $\epsilon_t$ is i.i.d. guassian error. The model functions as a linear regression with two nonlinear features, these being:

$$
h_t =\sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_1}\right) \, \text{and} \, \,
g_t =\sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_2}\right),
$$
latent representations of athletic "fitness" and "fatigue," respectively. These complicated features are convolutions of athletic training history with exponential decay but differ in the decay rate. The first convolution sum, $h_t$ typically has a much longer decay and represents fitness. Fatigue, or $g_t$ is much more transient and associated with a faster exponential decay. The regression coefficient on fatigue is typically much larger than for fitness as a counterpoint to its transience.

Simple math (done here) show that fitness and fatigue can be put into dynamic form as so:
$$
\begin{aligned}
h_t &= \theta_1 h_{t - 1} + \theta_1 w_{t - 1}, \\
g_t &= \theta_2 g_{t - 1} + \theta_2 w_{t - 1},
\end{aligned}
$$
where $\theta_1 =  e^{-1 / \tau_1}$ and $\theta_2 = e^{-1 / \tau_2}$.
This is a starting point to place this model within a Kalman Filter framework, which works quite well (link). Other approaches to fitting the model are to use brute force nonlinear least squares (link needed) and to use a distributed lag approach with a flexible functional form (link needed). Those methods work well also, but they are complex, and here I pose the central question of the article: *what would happen if you just OLS regressed on lagged variables of performance and training? How far could you get?*

The answer is "pretty far," but there are caviats, and blindly applying OLS regression on lagged variables will never get you to the truth.

## Rational transfer functions to the rescue

If we plug the bivariate dynamic representation of fitness and fatigue into the original model, we arrive at
$$
p_t = \mu + k_1 \theta_1 h_{t - 1} - k_2 \theta_2 g_{t - 1} + (\theta_1 + \theta_2) w_{t - 1} + \epsilon_t,
$$
which looks nicer but does us little good since $h_t$ and $g_t$ are unavailable to the modeler. Working a little harder, we can use the "backshift operator" $\text{B}$ defined by $\text{B} y_t = y_{t-1}$ for arbitrary time-indexed variable $y$, and arrive at 
$$
\begin{aligned}
(1 - \theta_1 \text{B}) h_t &= \theta_1 \text{B} w_t, \\
(1 - \theta_2 \text{B}) g_t &= \theta_2 \text{B} w_t.
\end{aligned}
$$
Solving for $h_t$ and $g_t$ and plugging back into the original model, we arrive at
$$
p_t = \mu + k_1 \frac{\theta_1 \text{B}}{1 - \theta_1\text{B}} w_t - k_2 \frac{\theta_2 \text{B}}{1 - \theta_2\text{B}} w_t + \epsilon_t.
$$
Thus we have two rational transfer functions operating on the exogenous input series $w_t$ (the training load that comes from the coach!). With rational transfer functions, denomonator terms of the form $(1 - \theta \text{B})$ correspond to an autoregressive impulse response, i.e., a process with a long memory, and this is a nuissance to us. There is an option to rid ourselves of the denomonator component, but not one without a cost.

A "common filter," as discussed in [Identification of Multiple-Input Transfer Function Models](https://www.researchgate.net/publication/276953549_Identification_of_Multiple-Input_Transfer_Function_Models) by Liu & Hanssens (1982) premultiplies the right and left-hand side of a time-series equation by $(1 - \theta \text{B})$. It does not change the transfer function weights, so you can apply multiple common filters in succession, and besides causing complexity and losing some rows due to lags, you have not destroyed the relationship between the input and output series.

The point is, if we were to use the common filter $(1 - \theta_1 \text{B}) (1 - \theta_2 \text{B})$, we would be rid of the autoregressive components. Let's see.

$$
\begin{aligned}
(1 - \theta_1 \text{B}) (1 - \theta_2 \text{B}) p_t = &(1 - \theta_1 \text{B}) (1 - \theta_2 \text{B}) \mu + k_1 \theta_1 (\text{B} - \theta_2 \text{B}^2) w_t - \\  &k_2 \theta_2 (\text{B} - \theta_1 \text{B}^2) w_t +  (1 - \theta_1 \text{B}) (1 - \theta_2 \text{B})\epsilon_t.
\end{aligned}
$$
It still looks ugly, but after expanding the polynomials applying the backshift operations, we arrive at:

$$
p_t = \mu^* + (\theta_1 + \theta_2) p_{t - 1} - \theta_1 \theta_2 p_{t - 2} + (k_1\theta_1 - k_2 \theta_2) w_{t-1} - \theta_1 \theta_2 (k_1 - k_2) w_{t-2} + \eta_t,
$$
$$
\text{where} \,\, \mu^* = (1 - \theta_1) (1 - \theta_2) \mu \,\, \text{and} \,\, \eta_t = \epsilon_t - (\theta_1 + \theta_2) \epsilon_{t-1} + \theta_1\theta_2 \epsilon_{t-2}.
$$

This is just a regression of performance on two lagged values of itself and of the external training input, almost. There is the issue of the MA(2) error structure that was induced by the common filter. We will see that this has real consequences.


Good picture:
https://en.wikipedia.org/wiki/Dynamic_Bayesian_network#/media/File:R%C3%A9seau_bay%C3%A9sien_simplifi%C3%A9.svg

## Example in R

This section will use simulated data that can be reproduced from an [R gist](https://gist.github.com/baogorek/6d682e42079005b3bde951e98ebae89e), or downloaded directly as a [csv file](https://drive.google.com/open?id=1kk40wiVYzPXOkrPffU55Vzy-LLTrgAVh). To run the R code below, change the file path in the following R code block:

In [2]:
train_df <- read.csv("/mnt/c/devl/data/train_df.csv")
head(train_df)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘nlme’

The following object is masked from ‘package:dplyr’:

    collapse



day,day_of_week,period,w,perf
1,0,build-up,10,489.1974
2,1,build-up,40,500.5453
3,2,build-up,42,479.8866
4,3,build-up,31,474.2269
5,4,build-up,46,459.3228
6,5,build-up,20,467.1399


Before running any regressions, the following use of R as a calculator shows the theoretical values of all the coefficients in the lagged variable model:

In [5]:
# True parameter values:
mu <- 496
theta1 <- exp(-1 / 60)
theta2 <- exp(-1 / 13)
k1 <- .07
k2 <- .27

#Theoretical coefficient for intercept is
(1 - theta1) * (1 - theta2) * mu

#Theoretical coefficient for performance lagged once is
theta1 + theta2

#Theoretical coefficient for performance lagged twice is
-theta1 * theta2

#Theoretical coefficient for training lagged once is
k1 * theta1 - k2 * theta2

#Theoretical coefficient for training lagged twice is
-theta1 * theta2 * (k1 - k2)

[1] 0.6069828

[1] 1.909433

[1] -0.9106563

[1] -0.1811665

[1] 0.1821313

Thus, the goal is to recover the following equation via lagged variable regression:

$$
\begin{aligned}
&p_t = .6070 + 1.9094 p_{t - 1} - .9107 p_{t - 2} - .1812 w_{t-1} + .1821 w_{t-2} + \eta_t, \\
&  \text{where}\, \, \eta_t = \epsilon_t - 1.9094 \epsilon_{t-1} + .9107 \epsilon_{t-2}.
\end{aligned}
$$ Note that the roots of the characteristic polynomial for both the AR component and MA component is
$$
1 - 1.9094 x + .9107 x^2 = 0
$$
which has real roots of 1.08 and 1.02, both larger than 1; the AR component is stationary and the MA component is invertible, **barely**. Still, given that the performance is being driven by external training factors, the model still depends heavily on where the athelete is in time, i.e., $t$, and hence the model unconditionally is still nonstationary. But it is conditionally stationary given $w$.

## Regression time
To prepare for the regression, the code below lags performance and training variables once and twice each. A shout out is in order to dplyr for making this so easy.

In [7]:
library(dplyr)

train_aug <- train_df %>%
  mutate(perf_lag1 = lag(perf, n = 1, order_by = day),
         perf_lag2 = lag(perf, n = 2, order_by = day),
         train_lag1 = lag(w, n = 1, order_by = day),
         train_lag2 = lag(w, n = 2, order_by = day))

When I think about OLS regression with lagged variables, I usually am not that concerned. I remember, for instance, that the OLS regression estimate for an AR(1) model is strongly consistent. It is not fully efficient because a regressor is correlated with *past* residuals, but it's still consistent. So let's see what happens when we regress performance on these lagged variables using OLS.

In [9]:
my_lm <- lm(perf ~ perf_lag1 + perf_lag2 + train_lag1 + train_lag2,
            data = train_aug[3:nrow(train_aug), ])
summary(my_lm)


Call:
lm(formula = perf ~ perf_lag1 + perf_lag2 + train_lag1 + train_lag2, 
    data = train_aug[3:nrow(train_aug), ])

Residuals:
     Min       1Q   Median       3Q      Max 
-21.0297  -6.3896   0.3202   5.7529  25.3549 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 33.99821   12.09470   2.811 0.005327 ** 
perf_lag1    0.48066    0.05619   8.553 1.18e-15 ***
perf_lag2    0.46189    0.05504   8.393 3.45e-15 ***
train_lag1  -0.15602    0.04406  -3.541 0.000475 ***
train_lag2  -0.02346    0.04516  -0.520 0.603807    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.518 on 252 degrees of freedom
Multiple R-squared:  0.9117,	Adjusted R-squared:  0.9103 
F-statistic: 650.4 on 4 and 252 DF,  p-value: < 2.2e-16


Ok, what's going on here? There is clearly bias, and substantial bias at that, in these estimates. This leads to a hugely imporant point: *OLS regression is consistent for dynamic regressions when there is no serial correlation remaining in the residuals.* The world "remaining" is somewhat redundant, because they are residuals, but I use it to emphasise that while the AR and MA components of a traditional ARMA model are meant to remove serial correlation from the residuals (i.e., "whitening"), if you didn't do a perfect job, there will be bias. This is because the lagged regressors are now *contemporanesly correlated* with the regression error - the definition of "endogeneity."

In this case, we were guaranteed to have serial correlation in the residuals because of our common filter operation. The common filter gave us a nice reduced form; now we have to pay the piper. Fortunately, R's `nlme` package offers generalized least squares model fitting via the `gls` function, which handle our (barely invertible) MA error structure out of the box.

In [10]:
library(nlme)
my_gls <- gls(perf ~ perf_lag1 + perf_lag2 + train_lag1 + train_lag2,
              data = train_aug[3:nrow(train_aug), ],
              corARMA(form = ~day, p = 0, q = 2))
summary(my_gls)

Generalized least squares fit by REML
  Model: perf ~ perf_lag1 + perf_lag2 + train_lag1 + train_lag2 
  Data: train_aug[3:nrow(train_aug), ] 
       AIC      BIC    logLik
  1804.258 1832.493 -894.1288

Correlation Structure: ARMA(0,2)
 Formula: ~day 
 Parameter estimate(s):
    Theta1     Theta2 
-1.9059497  0.9117409 

Coefficients:
                 Value  Std.Error    t-value p-value
(Intercept)  0.6571088 0.11700730    5.61596       0
perf_lag1    1.9187158 0.00815689  235.22646       0
perf_lag2   -0.9200058 0.00815495 -112.81568       0
train_lag1  -0.1662026 0.02238219   -7.42566       0
train_lag2   0.1664704 0.02241510    7.42671       0

 Correlation: 
           (Intr) prf_l1 prf_l2 trn_l1
perf_lag1   0.054                     
perf_lag2  -0.076 -1.000              
train_lag1 -0.156  0.357 -0.352       
train_lag2  0.115 -0.372  0.368 -0.999

Standardized residuals:
          Min            Q1           Med            Q3           Max 
-2.6183709992 -0.6998483434  0.000130

Note that we've now recovered the true parameter values, at least up to plausible estimation error.